In [1]:
%load_ext autoreload
%autoreload 2

import os

while "notebooks" in os.getcwd():
    os.chdir("..")

from datasets import load_dataset
from numpy import array_equal
from sklearn.metrics import mean_squared_error

from mim_nlp.regressor import AutoRegressor

/opt/anaconda/anaconda3/envs/mim_nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading an open-source dataset

In [2]:
dataset = load_dataset("imdb")

Found cached dataset imdb (/home/portal/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 420.00it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

# Training the model on the dataset

In [4]:
model = AutoRegressor(
    batch_size=4,
    learning_rate=1e-3,
    epochs=4,
    pretrained_model_name_or_path="bert-base-uncased",
    input_size=512,
    device="cuda",
    many_gpus=True,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
model.fit(dataset["train"]["text"], dataset["train"]["label"])

/opt/anaconda/anaconda3/envs/mim_nlp/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/anaconda/anaconda3/envs/mim_nlp/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1563,0.309400
3126,0.277500
4689,0.263500
6252,0.253700


# Get predictions on the test set

In [6]:
predictions = model.predict(dataset["test"]["text"])

# Calculate the accuracy score

In [7]:
mean_squared_error(dataset["test"]["label"], predictions)

0.2505673680812737

# Saving the model

In [8]:
model.save("models/auto_regressor")

# Loading the model

In [9]:
model_loaded = AutoRegressor.load("models/auto_regressor")

In [10]:
predictions_from_loaded = model.predict(dataset["test"]["text"])

In [11]:
assert array_equal(predictions, predictions_from_loaded)